In [28]:
import numpy as np
from collections import defaultdict
import xgboost as xgb
import gtnlplib.preproc
import gtnlplib.constants
from nltk.tokenize import word_tokenize, sent_tokenize
import cPickle as cp

In [6]:
reload(gtnlplib.preproc)

gtnlplib.preproc.docsToBOWs(gtnlplib.constants.TRAINKEY)
gtnlplib.preproc.docsToBOWs(gtnlplib.constants.DEVKEY)
all_tr_insts,all_dev_insts= gtnlplib.preproc.loadInstances(gtnlplib.constants.TRAINKEY, gtnlplib.constants.DEVKEY)
ac_train = gtnlplib.preproc.getAllCounts(gtnlplib.preproc.dataIterator(gtnlplib.constants.TRAINKEY))


In [9]:
ac_train[gtnlplib.constants.OFFSET] = 0
a = sorted(ac_train.items(), key=lambda x:x[1], reverse=True)
word_idx = {}
for i in range(len(a)):
    word_idx[a[i][0]] = i

In [13]:
def get_XY(filekey):
    idx_dict = {'POS' : 0, 'NEG' : 1, 'NEU' : 2}
    X = []
    Y = []
    with open(filekey, 'r') as f_idxes:
        root_folder = filekey.split('/')[0]
        for name_label in f_idxes:
            filename, label = name_label.split(' ')
            filename = root_folder + '/' + filename
            label = label.strip()
            with open(filename.strip(), 'r') as f:                
                cur_seq = []
                Y.append(idx_dict[label])
                for line in f:
                    decoded = line.decode('ascii','ignore')
                    sentences = sent_tokenize(decoded)           
                    for s in sentences:
                        words = word_tokenize(s)        
                        for w in words:
                            w = w.lower()
                            if w in word_idx:
                                cur_seq.append(word_idx[w])
                X.append(cur_seq)
    return X, Y

trainX, trainY = get_XY(gtnlplib.constants.TRAINKEY)
devX, devY = get_XY(gtnlplib.constants.DEVKEY)


In [29]:
out_file = open('lstm/amazon.pkl', 'wb')
out_dict = {'train' : (trainX, trainY), 'dev' : (devX, devY)}
cp.dump(out_dict, out_file, cp.HIGHEST_PROTOCOL)
out_file.close()